For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

In [233]:
#Install relevant libraries for scraping

!pip install bs4
!pip install requests

from bs4 import BeautifulSoup
import requests

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [270]:
import pandas as pd
pd.set_option('display.max_rows', 500)

#URL of Toronto Wiki.
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# get the contents of the webpage in text format and store in a variable called data
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

tables = soup.find_all('table', rules="all")

In [271]:
toronto_data = pd.DataFrame(columns=["PostalCode", "Combined"])

for row in tables[0].tbody.find_all("tr"):
    for col in row.find_all("td"):
        postal = col.find('br').previousSibling
        span = col.find_all("span")
        
        PostalCode = postal
        Combined = span[0].text.strip().strip(')').replace(' /',',').replace(')',' ').strip(' ')
        toronto_data = toronto_data.append({"PostalCode":PostalCode, "Combined":Combined}, ignore_index=True)

In [272]:
toronto_data = toronto_data[~(toronto_data['Combined'].str.contains("Not"))]

In [273]:
splitData = toronto_data["Combined"].str.split("(", n=1, expand=True)

In [274]:
toronto_data["Borough"] = splitData[0]
toronto_data["Neighborhood"] = splitData[1].str.strip(")")

In [275]:
# toronto_data.drop(columns=['Combined'], inplace=True)
toronto_data.reset_index(drop=True, inplace=True)
toronto_data['Borough']=toronto_data['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto_data['Neighborhood']=toronto_data['Neighborhood'].replace({'Don Mills South(Flemingdon Park':'Don Mills South','Downsview East  (CFB Toronto':'Downsview East'})

toronto_data

,PostalCode,Combined,Borough,Neighborhood
0,M3A,North York(Parkwoods,North York,Parkwoods
1,M4A,North York(Victoria Village,North York,Victoria Village
2,M5A,"Downtown Toronto(Regent Park, Harbourfront",Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,"North York(Lawrence Manor, Lawrence Heights",North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park(Ontario Provincial Government,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke(Islington Avenue,Etobicoke,Islington Avenue
6,M1B,"Scarborough(Malvern, Rouge",Scarborough,"Malvern, Rouge"
7,M3B,North York(Don Mills North,North York,Don Mills North
8,M4B,"East York(Parkview Hill, Woodbine Gardens",East York,"Parkview Hill, Woodbine Gardens"
9,M5B,"Downtown Toronto(Garden District, Ryerson",Downtown Toronto,"Garden District, Ryerson"
